In [1]:
import cv2
import math
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator

cv2.namedWindow("Prediksi Video", cv2.WINDOW_NORMAL)

jenis_kendaraan = { 0: 'Mobil', 1: 'Motor'}
model = YOLO('best.pt')
model.fuse()
# blob:http://103.71.191.168/507c9656-d9dc-4e1e-9ea4-383ceeac7521
# lokasiVideo = "http://103.71.191.168/8888/player.html"

Model summary (fused): 168 layers, 11126358 parameters, 0 gradients


In [2]:
while True:
    results = model.predict(lokasiVideo, conf=0.26, iou=0.8, stream=True)
    print(results)
    print(type(results))
    annotated_image = results[0].plot()

    cv2.namedWindow("Prediksi Video", cv2.WINDOW_FREERATIO)
    cv2.imshow("Prediksi Video", annotated_image)

    if cv2.waitKey(1) & 0xFF == ord("q"):
            break

<generator object BasePredictor.stream_inference at 0x000001E9EC7F4580>
<class 'generator'>


TypeError: 'generator' object is not subscriptable

In [3]:
results = model.predict(lokasiVideo, conf=0.26, iou=0.8, stream=True)
print(results)
print(type(results))

<generator object BasePredictor.stream_inference at 0x000001E9ECB76CF0>
<class 'generator'>


In [4]:
print(results)

TypeError: 'generator' object is not subscriptable

In [6]:
# Lokasi stream video (ubah ke stream yang sesuai)
lokasiVideo = "http://103.71.191.168/8888/player.html"

# Jendela untuk menampilkan hasil
cv2.namedWindow("Prediksi Video", cv2.WINDOW_NORMAL)

# Infinite loop untuk memproses stream video
try:
    for path, im, pred in model.predict(lokasiVideo, conf=0.26, iou=0.8, stream=True):
        # Konversi BGR ke RGB
        im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)

        # Pred adalah tensor Nx6 dengan struktur [xyxy, conf, cls]
        for *xyxy, conf, cls in reversed(pred):
            label = f'{model.names[int(cls)]} {conf:.2f}'
            Annotator(im, line_width=2, example=str(model.names)).box_label(xyxy, label, color=(255, 0, 0))

        # Tampilkan gambar
        cv2.imshow("Prediksi Video", im[:, :, ::-1])  # Konversi RGB ke BGR untuk OpenCV

        # Break loop dengan menekan 'q'
        if cv2.waitKey(1) == ord('q'):
            break

except KeyboardInterrupt:
    pass

# Release resources
cv2.destroyAllWindows()

ConnectionError: 1/1: http://103.71.191.168/8888/player.html... Failed to open http://103.71.191.168/8888/player.html

In [1]:
import cv2
import numpy as np

# Replace with your stream URL
# lokasiVideo = "http://103.71.191.168/8888/player.html"
STREAM_URL = "http://103.71.191.168:8080/player.html"

def stream_video(url):
    # Open a connection to the stream
    cap = cv2.VideoCapture(url)

    if not cap.isOpened():
        print("Error: Could not open video stream.")
        return

    try:
        # Loop to continuously get frames
        while True:
            # Read a new frame
            ret, frame = cap.read()
            if not ret:
                print("Error: Can't receive frame (stream end?). Exiting ...")
                break

            # Display the frame
            cv2.imshow('Video Stream', frame)

            # Press 'q' to close the window
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    finally:
        # Release the VideoCapture object and close windows
        cap.release()
        cv2.destroyAllWindows()

if __name__ == '__main__':
    stream_video(STREAM_URL)

Error: Could not open video stream.


In [7]:
import requests
import time
import os

# Replace these with the appropriate values
base_url = "http://103.71.191.168/8888/hls/live.stream-{}.ts"
destination_folder = "downloaded_ts"  # Folder where you want to save the .ts files

# Ensure the destination folder exists
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

# Function to download a single .ts file
def download_ts_file(url, destination_path):
    try:
        response = requests.get(url, stream=True)
        if response.status_code == 200:
            with open(destination_path, 'wb') as file:
                for chunk in response.iter_content(chunk_size=1024):
                    if chunk:  # filter out keep-alive new chunks
                        file.write(chunk)
            print(f"Downloaded: {destination_path}")
        else:
            print(f"Failed to download: {url} (Status code: {response.status_code})")
    except Exception as e:
        print(f"An error occurred: {e}")

# Example of how to download files in a loop
# Adjust the loop as needed based on how you update the timestamp 'X'
i = 0
try:
    while True:
        current_timestamp = int(time.time()) # Get current timestamp
        file_name = f"live.stream-{current_timestamp}.ts"
        ts_url = base_url.format(current_timestamp)
        destination_path = os.path.join(destination_folder, file_name)
        
        download_ts_file(ts_url, destination_path)
        
        # Wait for a bit before downloading the next file
        # Adjust the sleep time as needed
        time.sleep(10)
        i+=20
except KeyboardInterrupt:
    print("Stopped by the user.")

Failed to download: http://103.71.191.168/8888/hls/live.stream-257815.ts (Status code: 404)
Failed to download: http://103.71.191.168/8888/hls/live.stream-257845.ts (Status code: 404)
Stopped by the user.


In [25]:
import requests

# The URL from which you want to fetch data
url = 'http://103.71.191.168/8888/hls/live.stream.m3u8'

# Perform the GET request
response = requests.get(url)

# Make sure the request was successful
oldTs = 0
if response.status_code == 200:
    # Split the response text into lines
    lines = response.text.strip().split('\n')

    
    # Get the last line
    last_line = lines[-1] if lines else None

    currentUrl = f'http://103.71.191.168/8888/hls/{last_line}'

    print(currentUrl)
else:
    print(f"Failed to retrieve data: {response.status_code}")


http://103.71.191.168/8888/hls/live.stream-258212.ts


In [1]:
import cv2
import requests
import numpy as np

# The URL of the .ts file
url = 'http://103.71.191.168/8888/hls/live.stream-258212.ts'

# Send a GET request to the server
response = requests.get(url, stream=True)

# Check if the request was successful
if response.status_code == 200:
    # The bytes of the video stream
    video_bytes = bytes()

    # Read in chunks to avoid using too much memory
    for chunk in response.iter_content(chunk_size=1024):
        video_bytes += chunk

    with open('video.ts', 'wb') as file:
        file.write(video_bytes)
    
    exit()
    
    # Convert the bytes to a numpy array
    video_array = np.frombuffer(video_bytes, dtype=np.uint8)

    # Use OpenCV to decode the video stream
    video = cv2.imdecode(video_array, cv2.IMREAD_UNCHANGED)

    # Check if the video was successfully decoded
    if video is not None:
        # Here, you can process the video frame or display it
        cv2.imshow('Frame', video)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    else:
        print("Could not decode video stream.")
else:
    print(f"Failed to retrieve video: {response.status_code}")


Could not decode video stream.


: 